In [ ]:
!pip install torchtext==0.9.1

# >Reference: https://github.com/bentrevett/pytorch-sentiment-analysis

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import spacy
import torchtext
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.legacy import data
import torch.nn.functional as F
import gc


import matplotlib.pyplot as plt
import glob
import plotly.express as px
import warnings
import plotly as py
import plotly.graph_objs as go

import string
import re
import random
nlp = spacy.load('en_core_web_lg')

In [ ]:
tweetsDF = pd.read_csv("../input/covid19-vaccine-tweets-with-sentiment-annotation/covid-19_vaccine_tweets_with_sentiment.csv")

In [ ]:
tweetsDF.head()

In [ ]:
tweetsDF.info()

In [ ]:
def removeStopwords(text):
    doc = nlp(text)
    clean_text = ' '
    for txt in doc:
        if (txt.is_stop == False):
            clean_text = clean_text + " " + str(txt)        
    return clean_text

def removePunctuations(text):
    return text.translate(str.maketrans('', '', string.punctuation)).replace(" s ", " ")

def removeLinks(text):
    clean_text = re.sub('https?://\S+|www\.\S+', '', text)
    return clean_text

def removeNumbers(text):
    clean_text = re.sub(r'\d+', '', text)
    return clean_text
def removenewline(text):
    clean_text = re.sub(r'(\n+)', '', text)
    return clean_text


import emoji
#does the text contain an emoji?
def text_has_emoji(text):
    for character in text:
        if character in emoji.UNICODE_EMOJI:
            return True
    return False
#remove the emoji
def deEmojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

In [ ]:
def clean(text):
    if text_has_emoji(text):
        text=deEmojify(text)
    text = text.lower()
    text =removenewline(text)
    text = removeStopwords(text)
    text = removePunctuations(text)
    text = removeNumbers(text)
    text = removeLinks(text)
    return text

In [ ]:
tweetsDF['text_clean'] = tweetsDF['tweet_text'].apply(clean)
tweetsDF['label'] = tweetsDF['label'].map({1:0, 2:1, 3:2})# renumbering labels to avoid error in the one hot encoding process
tweetsDF.drop("tweet_text",axis=1).to_csv('mycsvfile.csv',index=False)

In [ ]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
np.random.seed(SEED)

#TEXT = data.Field(tokenize = 'spacy',
                #tokenizer_language = 'en_core_web_sm',
                 #lower = True)

#LABEL = data.LabelField(dtype=torch.int64)

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
len(tokenizer.vocab)

In [ ]:
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')

print(tokens)

In [ ]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

In [ ]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

In [ ]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [ ]:
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [ ]:
fields = [("tweet_id",None), ('label',LABEL),('text_clean', TEXT)]

In [ ]:
twitterDataset = data.dataset.TabularDataset(
        path="./mycsvfile.csv", 
        format="CSV", 
        fields=fields,
        #csv_reader_params={"quotechar": None,"delimiter":";"},
        skip_header=True)

In [ ]:
(train, test, valid) = twitterDataset.split(split_ratio=[0.6,0.2,0.2],
                                            stratified=True, strata_field='label',random_state = random.seed(SEED))

(len(train),len(test),len(valid))

In [ ]:
#MAX_VOCAB_SIZE = 14309

#TEXT.build_vocab(train, 
                 #max_size = MAX_VOCAB_SIZE, 
                 #vectors = "glove.6B.100d", 
                 #unk_init = torch.Tensor.normal_)
#LABEL.build_vocab(train)

In [ ]:
LABEL.build_vocab(train)

In [ ]:
print(LABEL.vocab.stoi)

In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test), 
    batch_size = BATCH_SIZE, 
    sort_within_batch=False,
    sort_key = lambda x: len(x.text_clean),
    device = device)

In [ ]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNNBert(nn.Module):
    def __init__(self,bert, n_filters, filter_sizes, output_dim, 
                 dropout):
        
        super().__init__()
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        #self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        #text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        with torch.no_grad():
            embedded = self.bert(text)[0]
        
        
        #embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [ ]:
#INPUT_DIM = len(TEXT.vocab)
#EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(LABEL.vocab)
DROPOUT = 0.5
#PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNNBert(bert,N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

In [ ]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [ ]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

In [ ]:
#pretrained_embeddings = TEXT.vocab.vectors

#model.embedding.weight.data.copy_(pretrained_embeddings)

In [ ]:
#UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

#model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
#model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [ ]:
def traindef(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        #print(batch.text_clean.shape)
        
        optimizer.zero_grad()
        
        predictions = model(batch.text_clean)
        
        batch.label  = torch.tensor(batch.label, dtype=torch.long, device=device)
        
        loss = criterion(predictions, batch.label)
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text_clean)
            
            batch.label  = torch.tensor(batch.label, dtype=torch.long, device=device)
            loss = criterion(predictions, batch.label)
            
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 50

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = traindef(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

In [ ]:
model.load_state_dict(torch.load('tut5-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

In [ ]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return torch.argmax(prediction.squeeze()).item()

In [ ]:
categories = {0: "Negative", 1:"Neutral",2: "Positive"}

In [ ]:
pred_class = predict_sentiment(model,tokenizer, "worst vaccine")
print(f'Predicted class is: {categories[int(LABEL.vocab.itos[pred_class])]}')

In [ ]:
pred_class = predict_sentiment(model,tokenizer, "best vaccine")
print(f'Predicted class is: {categories[int(LABEL.vocab.itos[pred_class])]}')